In [64]:
from pathlib import Path
import tifffile
import napari
from apoc import ObjectSegmenter, PixelClassifier
from skimage import measure
import pandas as pd
import os

In [65]:
# Define the directory containing your files
directory_path = Path("./data/before_sonication/")

# Initialize an empty list to store the images
images = []

# Iterate through the files in the directory
for file_path in directory_path.glob('*.TIF'):
    
    # Append the file to the corresponding group
    images.append(str(file_path))

In [66]:
# This is just a check, it prints the image filepaths contained in images
for filepath in images:
    print(filepath)

data\before_sonication\plate 10_Top Slide_TD_p00_0_A02f00d1.TIF
data\before_sonication\plate 11_Top Slide_TD_p00_0_A03f00d1.TIF
data\before_sonication\plate 12_Top Slide_TD_p00_0_A04f00d1.TIF
data\before_sonication\plate 13_Top Slide_TD_p00_0_B01f00d1.TIF
data\before_sonication\plate 14_Top Slide_TD_p00_0_B02f00d1.TIF
data\before_sonication\plate 15_Top Slide_TD_p00_0_B03f00d1.TIF
data\before_sonication\plate 16_Top Slide_TD_p00_0_B04f00d1.TIF
data\before_sonication\plate 17_Top Slide_TD_p00_0_A01f00d1.TIF
data\before_sonication\plate 18_Top Slide_TD_p00_0_A02f00d1.TIF
data\before_sonication\plate 19_Top Slide_TD_p00_0_A03f00d1.TIF
data\before_sonication\plate 1_Top Slide_TD_p00_0_A01f00d1.TIF
data\before_sonication\plate 20_Top Slide_TD_p00_0_A04f00d1.TIF
data\before_sonication\plate 21_Top Slide_TD_p00_0_B01f00d1.TIF
data\before_sonication\plate 22_Top Slide_TD_p00_0_B02f00d1.TIF
data\before_sonication\plate 23_Top Slide_TD_p00_0_B03f00d1.TIF
data\before_sonication\plate 24_Top Slide

In [67]:
# Create an empty dictionary to store filename:biofilm_area pairs and later convert it into a pandas df
biofilm_areas = {}

# Initialize Napari viewer instance
viewer = napari.Viewer(ndisplay=2)

for filepath in images:
    # Get the filename without the extension
    filename = Path(filepath).stem
    
    # Load one RGB image and extract the green channel for APOC
    rgb_img = tifffile.imread(filepath, is_ome = False)

    # Extract green channel [1]
    img = rgb_img[:,:,1]

    # Visualize greyscale green channel in napari
    viewer.add_image(img, name=f"Biofilm_{filename}")
    
    # Apply pretrained object segmenter from APOC
    # object_segmenter = ObjectSegmenter(opencl_filename="./pretrained_classifiers/ObjectSegmenter.cl")
    # obj_seg_result = object_segmenter.predict(image=img)
    # viewer.add_labels(obj_seg_result, name=f"Biofilm_objects_{filename}")
    
    # Apply pretrained semantic segmenter from APOC
    semantic_segmenter = PixelClassifier(opencl_filename="./pretrained_classifiers/PixelClassifier.cl")
    sem_seg_result = semantic_segmenter.predict(image=img)
    viewer.add_labels(sem_seg_result, name=f"Biofilm_layer_{filename}")
    
    # Calculate the area covered by the biofilm
    props = measure.regionprops(sem_seg_result) # Calculate the background and biofilm area
    
    # Extract area values
    area_values = [prop.area for prop in props]
    
    # Testing purposes
    print(f"Filename: {filename} Background: {area_values[0]} pixels, Biofilm: {area_values[1]} pixels")
    
    # Add new filename:biofilm_area pair to biofilm_areas
    biofilm_areas[filename] = area_values[1]


Filename: plate 10_Top Slide_TD_p00_0_A02f00d1 Background: 11401208.0 pixels, Biofilm: 1121344.0 pixels
Filename: plate 11_Top Slide_TD_p00_0_A03f00d1 Background: 12149476.0 pixels, Biofilm: 373076.0 pixels
Filename: plate 12_Top Slide_TD_p00_0_A04f00d1 Background: 11733624.0 pixels, Biofilm: 788928.0 pixels
Filename: plate 13_Top Slide_TD_p00_0_B01f00d1 Background: 12103655.0 pixels, Biofilm: 418897.0 pixels
Filename: plate 14_Top Slide_TD_p00_0_B02f00d1 Background: 11553813.0 pixels, Biofilm: 968739.0 pixels
Filename: plate 15_Top Slide_TD_p00_0_B03f00d1 Background: 11975417.0 pixels, Biofilm: 547135.0 pixels
Filename: plate 16_Top Slide_TD_p00_0_B04f00d1 Background: 12208637.0 pixels, Biofilm: 313915.0 pixels
Filename: plate 17_Top Slide_TD_p00_0_A01f00d1 Background: 7472772.0 pixels, Biofilm: 877692.0 pixels
Filename: plate 18_Top Slide_TD_p00_0_A02f00d1 Background: 12120170.0 pixels, Biofilm: 402382.0 pixels
Filename: plate 19_Top Slide_TD_p00_0_A03f00d1 Background: 12099344.0 pix

In [68]:
# Convert dictionary to pandas DataFrame
df = pd.DataFrame(list(biofilm_areas.items()), columns=['filename', 'biofilm_area'])

# Define the output folder path
output_folder = './output/'

# Check if the output folder exists, create it if not
try:
    os.makedirs(output_folder)
    print(f"Output folder created: {output_folder}")
except FileExistsError:
    print(f"Output folder already exists: {output_folder}")

# Save the DataFrame to a CSV file in the output folder
df.to_csv(os.path.join(output_folder, 'biofilm_areas.csv'), index=False)

# Display the resulting dataframe
df

Output folder created: ./output/


,filename,biofilm_area
0,plate 10_Top Slide_TD_p00_0_A02f00d1,1121344.0
1,plate 11_Top Slide_TD_p00_0_A03f00d1,373076.0
2,plate 12_Top Slide_TD_p00_0_A04f00d1,788928.0
3,plate 13_Top Slide_TD_p00_0_B01f00d1,418897.0
4,plate 14_Top Slide_TD_p00_0_B02f00d1,968739.0
5,plate 15_Top Slide_TD_p00_0_B03f00d1,547135.0
6,plate 16_Top Slide_TD_p00_0_B04f00d1,313915.0
7,plate 17_Top Slide_TD_p00_0_A01f00d1,877692.0
8,plate 18_Top Slide_TD_p00_0_A02f00d1,402382.0
9,plate 19_Top Slide_TD_p00_0_A03f00d1,423208.0


<h1> For development purpose (below) </h1>

In [ ]:
# Load one RGB image and extract the green channel for APOC
rgb_img = tifffile.imread(images[5], is_ome = False)

# Extract green channel [1]
img = rgb_img[:,:,1]

# Visualize greyscale green channel in napari
viewer = napari.Viewer(ndisplay=2)
viewer.add_image(img, name="Biofilm")

<Image layer 'Biofilm' at 0x21a09bb7fd0>

In [ ]:
# Apply pretrained object segmenter from APOC
object_segmenter = ObjectSegmenter(opencl_filename="./pretrained_classifiers/ObjectSegmenter.cl")
obj_seg_result = object_segmenter.predict(image=img)
viewer.add_labels(obj_seg_result, name="Biofilm_objects")

<Labels layer 'Biofilm_objects' at 0x21a092eab20>

In [ ]:
# Apply pretrained semantic segmenter from APOC
semantic_segmenter = PixelClassifier(opencl_filename="./pretrained_classifiers/PixelClassifier.cl")
sem_seg_result = semantic_segmenter.predict(image=img)
viewer.add_labels(sem_seg_result, name="Biofilm_layer")

<Labels layer 'Biofilm_layer' at 0x21a082a2af0>

In [ ]:
# Calculate the area covered by the biofilm
props = measure.regionprops(sem_seg_result) # Calculate the background and biofilm area

# Extract area values
area_values = [prop.area for prop in props]

print(f"Background: {area_values[0]} pixels, Biofilm: {area_values[1]} pixels")

Background: 11975417.0 pixels, Biofilm: 547135.0 pixels
